In [1]:
import os
import pandas as pd

# Paths
source_folder = r"C:\This PC Folder\CA Tracker\Merging Sheets\Monthly CA Tracker"  # Folder containing 28 workbooks
output_file = "Monthly_CA_Tracker_Master_File.xlsx"  # Merged output file

# Step 1: Merge "main sheet" from all workbooks
merged_data = []  # List to store data from all sheets

for file_name in os.listdir(source_folder):
    if file_name.endswith(".xlsx"):
        file_path = os.path.join(source_folder, file_name)
        # Read the "main sheet" from each workbook
        try:
            df = pd.read_excel(file_path, sheet_name="Sheet1")
            df['Source File'] = file_name  # Optional: Add a column for source file
            merged_data.append(df)
        except Exception as e:
            print(f"Error reading {file_name}: {e}")

# Combine all data into a single DataFrame
if merged_data:
    merged_df = pd.concat(merged_data, ignore_index=True)

    try:
        # Step 2: Process the first pivot summary for 'Date'
        merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')  # Ensure Date is datetime format

        summary_table = merged_df.groupby(['Date', 'DA']).size().reset_index(name='Companies Processed')
        summary_table['Date'] = summary_table['Date'].dt.strftime('%m/%d/%Y')  # Convert Date to MM/DD/YYYY format

        pivot_summary = summary_table.pivot(index='DA', columns='Date', values='Companies Processed').fillna(0).astype(int)
        pivot_summary.columns.name = None  # Reset column headers to remove MultiIndex
        pivot_summary = pivot_summary[sorted(pivot_summary.columns, key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'), reverse=True)]
        pivot_summary['Total'] = pivot_summary.sum(axis=1)  # Add a Total column for each DA
        pivot_summary.loc['Total for Dates'] = pivot_summary.sum(axis=0)  # Add Total row

        # Step 3: Process the second pivot summary for 'Basic Info Date'
        if 'Basic Info Date' in merged_df.columns:
            merged_df['Basic Info Date'] = pd.to_datetime(merged_df['Basic Info Date'], errors='coerce')  # Ensure datetime format

            basic_info_summary_table = merged_df.groupby(['Basic Info Date', 'DA']).size().reset_index(name='Companies Processed')
            basic_info_summary_table['Basic Info Date'] = basic_info_summary_table['Basic Info Date'].dt.strftime('%m/%d/%Y')

            basic_info_pivot_summary = basic_info_summary_table.pivot(index='DA', columns='Basic Info Date', values='Companies Processed').fillna(0).astype(int)
            basic_info_pivot_summary.columns.name = None
            basic_info_pivot_summary = basic_info_pivot_summary[sorted(basic_info_pivot_summary.columns, key=lambda x: pd.to_datetime(x, format='%m/%d/%Y'), reverse=True)]
            basic_info_pivot_summary['Total'] = basic_info_pivot_summary.sum(axis=1)
            basic_info_pivot_summary.loc['Total for Dates'] = basic_info_pivot_summary.sum(axis=0)
        else:
            basic_info_pivot_summary = None
            print("Column 'Basic Info Date' not found in the data.")

        # Step 4: Create the Analyst Summary sheet (with the required metrics)
        # We add the condition to count "Yes" and "Announced" in the 'Is Corporate Action Found?' column, ignoring NaN (blank) values.
        analyst_summary = (
            merged_df.groupby('DA')
            .agg(
                Total_Companies_Assigned=('DA', 'size'),  # Count total companies assigned to each DA
                Processed_Companies=('Date', lambda x: x.notna().sum()),  # Count where Date is not NaN
                Yet_to_Process_Companies=('Date', lambda x: x.isna().sum()),  # Count where Date is NaN
                Corporate_Action_Found=('Is Corp Action found?', lambda x: (x.str.strip().str.lower() == 'yes').sum()),  # Count "Yes"
                Announced=('Is Corp Action found?', lambda x: (x.str.strip().str.lower() == 'announced').sum())  # Count "Announced"
            )
        )

        # Add total row at the bottom
        total_row = pd.DataFrame(
            analyst_summary.sum().rename("Total for Analysts")  # Rename the row
        ).T  # Convert to a DataFrame with a single row
        analyst_summary = pd.concat([analyst_summary, total_row], axis=0)  # Concatenate the total row

        # Step 5: Save all sheets to the Excel file
        with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
            # Save the merged data
            merged_df.to_excel(writer, index=False, sheet_name="Merged_Data")

            # Save the first pivot summary (Date)
            pivot_summary.to_excel(writer, sheet_name="Pivot_Date")

            # Save the second pivot summary (Basic Info Date) if it exists
            if basic_info_pivot_summary is not None:
                basic_info_pivot_summary.to_excel(writer, sheet_name="Pivot_Basic_Info")

            # Save the analyst summary sheet
            analyst_summary.to_excel(writer, sheet_name="Analyst_Summary")

        print(f"Merged data and pivot summaries saved to {output_file}")

    except KeyError as e:
        print(f"Error in creating summary tables: Missing column {e}")

else:
    print("No data to merge.")


c:\Users\aawhad\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\aawhad\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\aawhad\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\aawhad\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\aawhad\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\aawhad

Merged data and pivot summaries saved to Monthly_CA_Tracker_Master_File.xlsx
